In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

In [86]:
# Dataset class
class VeggieDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images  # Should be in (batch, channels, height, width)
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = np.transpose(self.images[idx], (1, 2, 0))  # Convert (C, H, W) -> (H, W, C)
        image = Image.fromarray((image * 255).astype(np.uint8))  # Convert to PIL Image
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.float32).squeeze()
        return image, label

# Model class
class VeggieVision(nn.Module):
    def __init__(self):
        super(VeggieVision, self).__init__()
        self.base_model = models.mobilenet_v2(pretrained=True)
        self.base_model.features.requires_grad = False  # Freeze base layers
        self.fc1 = nn.Linear(self.base_model.last_channel * 7 * 7, 128)  # Adjust input size here
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.base_model.features(x)
        x = x.reshape(x.size(0), -1)  # Use reshape
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  # Linear output for regression
        return x

# Data transformations
train_transform = transforms.Compose([
    transforms.RandomRotation(360),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Helper function to prepare data loaders
def prepare_data_loaders(X, y, validation_split=0.2, batch_size=32):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)
    train_dataset = VeggieDataset(X_train, y_train, transform=train_transform)
    val_dataset = VeggieDataset(X_val, y_val, transform=test_transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    return train_loader, val_loader

# Model initialization with scheduler setup
def initialize_model(device, learning_rate=1e-4):
    model = VeggieVision().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Set up a scheduler that reduces the learning rate on plateau of validation loss
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
    
    return model, criterion, optimizer, scheduler


# Training loop
def training_loop(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=20, print_freq=1):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Train the model
        train_loss = train(model, train_loader, criterion, optimizer, device, print_freq=print_freq)
        
        # Validate the model
        val_loss = validate(model, val_loader, criterion, device, print_freq=print_freq)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")
        
        # Step the scheduler based on the validation loss
        scheduler.step(val_loss)


# Fine-tuning setup
def setup_fine_tuning(model, optimizer, learning_rate=1e-5, n_unfreeze_layers=1):
    fine_tune(model, n_unfreeze_layers)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return optimizer

# Fine-tuning loop
def fine_tuning_loop(model, train_loader, val_loader, criterion, optimizer, device, fine_tune_epochs=2):
    for epoch in range(fine_tune_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        val_loss = validate(model, val_loader, criterion, device)
        print(f"Fine-tuning Epoch {epoch+1}/{fine_tune_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

def train(model, dataloader, criterion, optimizer, device, print_freq=1):
    model.train()
    running_loss = 0.0
    total_batches = len(dataloader)

    for batch_idx, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
        # Print batch loss every `print_freq` batches
        if (batch_idx + 1) % print_freq == 0:
            print(f"Batch {batch_idx+1}/{total_batches}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate(model, dataloader, criterion, device, print_freq=1):
    model.eval()
    running_loss = 0.0
    total_batches = len(dataloader)
    
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            running_loss += loss.item() * images.size(0)
            
            # Print batch loss every `print_freq` batches
            if (batch_idx + 1) % print_freq == 0:
                print(f"Validation Batch {batch_idx+1}/{total_batches}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss


# Fine-tuning function
def fine_tune(model, n_unfreeze_layers=1):
    layers = list(model.base_model.features.children())
    for layer in layers[-n_unfreeze_layers:]:
        for param in layer.parameters():
            param.requires_grad = True

def predict(model, X, device):
    model.eval()  # Set model to evaluation mode
    
    # Ensure X is a torch tensor with float32 data type and the correct shape
    if isinstance(X, np.ndarray):
        X = torch.tensor(X, dtype=torch.float32)
    
    # Add a batch dimension if predicting on a single image
    if len(X.shape) == 3:  # If X is (channels, height, width), add batch dimension
        X = X.unsqueeze(0)
    
    # Send the batch of images to the specified device
    X = X.to(device)
    
    with torch.no_grad():
        outputs = model(X)
        predictions = outputs.squeeze().cpu().numpy()  # Convert to numpy for easier handling

    return predictions


from sklearn.metrics import mean_squared_error
import numpy as np
import torch

def evaluate(model, X_test, y_test, device):
    model.eval()  # Set model to evaluation mode
    
    # Generate predictions for the entire test set using the predict function
    predictions = predict(model, X_test, device)
    
    # Calculate Mean Squared Error using the true labels and predictions
    mse = mean_squared_error(y_test, predictions)
    print(f'Mean Squared Error on Test Set: {mse:.4f}')
    
    return mse, np.array(predictions)


In [87]:
# Load the data
X = np.load("../data/processed_data/X.npy")
y = np.load("../data/processed_data/y.npy")

# Convert X from (batch, height, width, channel) to (batch, channel, height, width)
X = np.transpose(X, (0, 3, 1, 2)).astype(np.float32)
y = y.reshape(-1).astype(np.float32)

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [88]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
# Prepare data loaders
train_loader, val_loader = prepare_data_loaders(X_train, y_train, 0.2, 32)
    
# Initialize model, loss function, and optimizer
model, criterion, optimizer, scheduler = initialize_model(device, 1e-3)
    

c:\Users\juliu\.conda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\juliu\.conda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\juliu\.conda\lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [93]:
# Training phase
training_loop(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs = 2)

Epoch 1/2
Batch 1/15, Loss: 8410.4902
Batch 2/15, Loss: 11357.5312
Batch 3/15, Loss: 4908.5093
Batch 4/15, Loss: 2981.1782
Batch 5/15, Loss: 3605.3391
Batch 6/15, Loss: 6797.9438
Batch 7/15, Loss: 3585.3994
Batch 8/15, Loss: 1787.7040
Batch 9/15, Loss: 2068.0012
Batch 10/15, Loss: 2467.5237
Batch 11/15, Loss: 216807.7500
Batch 12/15, Loss: 4716.5762
Batch 13/15, Loss: 3977.4531
Batch 14/15, Loss: 4598.2803
Batch 15/15, Loss: 1248.3616
Validation Batch 1/4, Loss: 2844.3254
Validation Batch 2/4, Loss: 2243.9270
Validation Batch 3/4, Loss: 2579.0349
Validation Batch 4/4, Loss: 2285.7124
Epoch 1/2, Train Loss: 19104.8150, Validation Loss: 2507.2919
Epoch 2/2
Batch 1/15, Loss: 1672.2075
Batch 2/15, Loss: 2899.3528
Batch 3/15, Loss: 3853.6045
Batch 4/15, Loss: 2941.9451
Batch 5/15, Loss: 4671.2749
Batch 6/15, Loss: 3400.8130
Batch 7/15, Loss: 2164.2004
Batch 8/15, Loss: 3621.0308
Batch 9/15, Loss: 192725.0625
Batch 10/15, Loss: 5711.9707
Batch 11/15, Loss: 7732.3892
Batch 12/15, Loss: 7409.9

In [94]:
# Fine-tuning phase
#optimizer = setup_fine_tuning(model, optimizer, learning_rate=1e-5, n_unfreeze_layers=1)
#fine_tuning_loop(model, train_loader, val_loader, criterion, optimizer, device, 2)

In [96]:
# Evaluation phase
evaluate(model, X_test, y_test, device)

Mean Squared Error on Test Set: 4503.1807


(4503.1807,
 array([ 85.92173 , 113.28998 ,  73.26694 , 134.63368 , 138.56113 ,
         91.371   ,  82.55229 , 125.32519 , 118.35986 ,  91.33328 ,
        115.21034 ,  91.2572  ,  92.06623 , 115.58621 , 123.13333 ,
        113.234406,  87.81117 , 153.18767 ,  79.40527 ,  87.22033 ,
        102.14139 ,  93.86496 , 145.17398 , 114.56839 ,  77.53392 ,
         96.780785,  91.20685 , 135.11786 ,  99.32752 , 100.36951 ,
         85.12936 , 109.99549 , 113.77382 ,  84.85522 ,  97.80151 ,
        140.19856 ,  89.58275 , 106.668175, 103.25375 ,  86.25515 ,
         88.3547  , 131.36534 , 105.71373 ,  88.763466,  98.011765,
         88.961716, 108.523575,  94.915726,  83.93364 , 110.656494,
        132.17863 ,  80.69373 , 101.839035, 103.32461 ,  92.53502 ,
         83.64021 , 112.166275, 100.4583  , 141.89357 ,  82.50578 ,
         87.4367  ,  89.91699 , 118.32826 ,  94.778656,  85.295   ,
         93.02997 ,  79.36111 ,  88.74667 ,  94.9993  ,  99.486534,
         93.77359 , 100.85672 ,  87.